## PPP model implementation from scratch

### imports

In [1]:
import openfermion as of
from openfermion.ops.operators import BosonOperator, FermionOperator
from openfermion.utils.indexing import down_index, up_index
from openfermion.hamiltonians.special_operators import number_operator

In [64]:
def ppp(n_sites,
        U,
        unit_dist,
        z=1,
        verbose=False):
    n_spin_orbitals = 2*n_sites
    ppp_model = FermionOperator()
    
    for i in range(n_sites):
        print(i)
        # Add hopping terms.
        if i != n_sites:
            j = i+1
            hopping = get_hopping_term(unit_dist, n_spin_orbitals, i, j)
            if verbose:
                print(f"hopping term: {hopping}")
                print("*"*20)
            ppp_model += hopping
        
        # Add onsite terms.
        onsite = get_onsite_term(U, n_spin_orbitals, i)*0.5
        if verbose:
            print(f"onsite term: {onsite}")
            print("*"*20)
        ppp_model += onsite

       # Add intersite terms
        for j in range (n_sites):
            if i != j:
                intersite = get_intersite_term(unit_dist, U, n_spin_orbitals, i, j, z)
            #if verbose: print(f"intersite: {intersite}")
                ppp_model += intersite
#         if verbose: print("*"*40)


    return(ppp_model)


In [65]:
def get_hopping_term(unit_dist, n_sites, i, j):
    coefficient = get_tij(unit_dist, i, j)
    op_class = FermionOperator
    hopping_term = op_class(((i, 1), (j, 0)), coefficient)
    hopping_term += op_class(((j, 1), (i, 0)), coefficient.conjugate())
    return hopping_term

def get_tij(unit_dist, i, j):
    rij = (j-i)*unit_dist
#     tij = -2.4*(1-(rij-1.397)/rij)
    tij = -2.4
    return tij

def get_onsite_term(U, n_sites, i):
    op_class = FermionOperator
    n_i = number_operator(n_sites, i)
    print(n_i)
    onsite = U*n_i*(n_i-1)
    return onsite

def get_intersite_term(unit_dist, U, n_sites, i, j, z=1):
    vij = ohno(unit_dist, U, i, j)
    ni, nj = get_n(n_sites, z, i, j)
    intersite = vij*ni*nj
    return intersite

def ohno(unit_dist, U, i, j):
    rij = (j-i)*unit_dist
    vij = 14.397*(((28.794/2*U)**2)+rij**2)**0.5
    return vij

def get_n(n_sites, z, i, j):
    op_class = FermionOperator
    n_i = number_operator(n_sites, i)
    n_j = number_operator(n_sites, j)
    return (n_i-z, n_j-z)

In [66]:
n_sites = 2
U = 11.26
unit_dist = 1.33
ppp_model = ppp(n_sites, U, unit_dist)
print(ppp_model)

0
1.0 [0^ 0]
1
1.0 [1^ 1]
4667.958767951282 [] +
-4673.5887679512825 [0^ 0] +
5.63 [0^ 0 0^ 0] +
2333.979383975641 [0^ 0 1^ 1] +
-2.4 [0^ 1] +
-2.4 [1^ 0] +
-4673.5887679512825 [1^ 1] +
2333.979383975641 [1^ 1 0^ 0] +
5.63 [1^ 1 1^ 1] +
-2.4 [1^ 2] +
-2.4 [2^ 1]


In [67]:
from openfermion.linalg import get_sparse_operator, get_ground_state
from openfermion.transforms import jordan_wigner
from openfermion.linalg import eigenspectrum

print(type(ppp_model))
# Get qubit operator under Jordan-Wigner.
jw_hamiltonian = jordan_wigner(ppp_model)
# jw_hamiltonian.compress()
# print("jw_hamiltonian")
# print(jw_hamiltonian)
print('')

jw_spectrum = eigenspectrum(jw_hamiltonian)
print("jw_eigenspectrum")
print(jw_spectrum)
print('')

qubit_matrix = get_sparse_operator(jw_hamiltonian)
# print("Qubit Matrix")
# print(qubit_matrix)

exc_energy = jw_spectrum[1]-jw_spectrum[0]
print ("excitation energy",exc_energy)
print('\nEnergy of the model is {} .'.format(
   get_ground_state(qubit_matrix)[0]))

<class 'openfermion.ops.operators.fermion_operator.FermionOperator'>

jw_eigenspectrum
[-3.39411255e+00 -2.40061673e+00 -2.27373675e-13 -1.13103971e-13
  2.39938279e+00  3.39411255e+00  4.66795877e+03  4.66796000e+03]

excitation energy 0.9934958155738283

Energy of the model is -3.3941125496954574 .


In [62]:
from openfermion.ops.operators import BosonOperator, FermionOperator
from openfermion.utils.indexing import down_index, up_index
from openfermion.hamiltonians.special_operators import number_operator


def fermi_ppp(z,
            unit_dist,
            x_dimension,
            y_dimension,
            tunneling,
            coulomb,
            chemical_potential=0.,
            magnetic_field=0.,
            periodic=True,
            spinless=False,
            particle_hole_symmetry=False):
   
    if spinless:
        return _spinless_fermi_hubbard_model(x_dimension, y_dimension,
                                             tunneling, coulomb,
                                             chemical_potential, magnetic_field,
                                             periodic, particle_hole_symmetry,z,unit_dist)
    else:
        return _spinful_fermi_hubbard_model(x_dimension, y_dimension, tunneling,
                                            coulomb, chemical_potential,
                                            magnetic_field, periodic,
                                            particle_hole_symmetry,z,unit_dist)


def _spinful_fermi_hubbard_model(x_dimension, y_dimension, tunneling, coulomb,
                                 chemical_potential, magnetic_field, periodic,
                                 particle_hole_symmetry,z,unit_dist):

    # Initialize operator.
    n_sites = x_dimension * y_dimension
    n_spin_orbitals = 2 * n_sites
    hubbard_model = FermionOperator()

    # Loop through sites and add terms.
    for site in range(n_sites):

        # Get indices of right and bottom neighbors
        right_neighbor = _right_neighbor(site, x_dimension, y_dimension,
                                         periodic)
        bottom_neighbor = _bottom_neighbor(site, x_dimension, y_dimension,
                                           periodic)

        # Avoid double-counting edges when one of the dimensions is 2
        # and the system is periodic
        if x_dimension == 2 and periodic and site % 2 == 1:
            right_neighbor = None
        if y_dimension == 2 and periodic and site >= x_dimension:
            bottom_neighbor = None

        # Add hopping terms with neighbors to the right and bottom.
        if right_neighbor is not None:
            hubbard_model += _hopping_term(up_index(site),
                                           up_index(right_neighbor), -tunneling)
            hubbard_model += _hopping_term(down_index(site),
                                           down_index(right_neighbor),
                                           -tunneling)
        if bottom_neighbor is not None:
            hubbard_model += _hopping_term(up_index(site),
                                           up_index(bottom_neighbor),
                                           -tunneling)
            hubbard_model += _hopping_term(down_index(site),
                                           down_index(bottom_neighbor),
                                           -tunneling)

        # Add local pair Coulomb interaction terms.
        hubbard_model += _coulomb_interaction_term(n_spin_orbitals,
                                                   up_index(site),
                                                   down_index(site), coulomb,
                                                   particle_hole_symmetry)

        # Add chemical potential and magnetic field terms.
        hubbard_model += number_operator(n_spin_orbitals, up_index(site),
                                         -chemical_potential - magnetic_field)
        hubbard_model += number_operator(n_spin_orbitals, down_index(site),
                                         -chemical_potential + magnetic_field)
        
        # Add intersite_interaction terms.
        hubbard_model += _intersite_interaction_term(unit_dist, coulomb, n_sites, z)
       
    return hubbard_model


def _spinless_fermi_hubbard_model(x_dimension, y_dimension, tunneling, coulomb,
                                  chemical_potential, magnetic_field, periodic,
                                  particle_hole_symmetry,z,unit_dist):

    # Initialize operator.
    n_sites = x_dimension * y_dimension
    hubbard_model = FermionOperator()

    # Loop through sites and add terms.
    for site in range(n_sites):

        # Get indices of right and bottom neighbors
        right_neighbor = _right_neighbor(site, x_dimension, y_dimension,
                                         periodic)
        bottom_neighbor = _bottom_neighbor(site, x_dimension, y_dimension,
                                           periodic)

        # Avoid double-counting edges when one of the dimensions is 2
        # and the system is periodic
        if x_dimension == 2 and periodic and site % 2 == 1:
            right_neighbor = None
        if y_dimension == 2 and periodic and site >= x_dimension:
            bottom_neighbor = None

        # Add terms that couple with neighbors to the right and bottom.
        if right_neighbor is not None:
            # Add hopping term
            hubbard_model += _hopping_term(site, right_neighbor, -tunneling)
            # Add local Coulomb interaction term
            hubbard_model += _coulomb_interaction_term(n_sites, site,
                                                       right_neighbor, coulomb,
                                                       particle_hole_symmetry)
            # Add intersite_interaction terms.
            hubbard_model += _intersite_interaction_term(unit_dist, coulomb, x_dimension, z)
        if bottom_neighbor is not None:
            # Add hopping term
            hubbard_model += _hopping_term(site, bottom_neighbor, -tunneling)
            # Add local Coulomb interaction term
            hubbard_model += _coulomb_interaction_term(n_sites, site,
                                                       bottom_neighbor, coulomb,
                                                       particle_hole_symmetry)
            # Add intersite_interaction terms.
            hubbard_model += _intersite_interaction_term(unit_dist, coulomb, n_sites, z)

        # Add chemical potential. The magnetic field doesn't contribute.
        hubbard_model += number_operator(n_sites, site, -chemical_potential)

    return hubbard_model


def _hopping_term(i, j, coefficient, bosonic=False):
    op_class = BosonOperator if bosonic else FermionOperator
    hopping_term = op_class(((i, 1), (j, 0)), coefficient)
    hopping_term += op_class(((j, 1), (i, 0)), coefficient.conjugate())
    return hopping_term


def _coulomb_interaction_term(n_sites,
                              i,
                              j,
                              coefficient,
                              particle_hole_symmetry,
                              bosonic=False):
    op_class = BosonOperator if bosonic else FermionOperator
    number_operator_i = number_operator(n_sites, i, parity=2 * bosonic - 1)
    number_operator_j = number_operator(n_sites, j, parity=2 * bosonic - 1)
#     print("Number_Operator")
#     print(number_operator_i)
    if particle_hole_symmetry:
        number_operator_i -= op_class((), 0.5)
        number_operator_j -= op_class((), 0.5)
    return coefficient * number_operator_i * (number_operator_j-1)

def _intersite_interaction_term(unit_dist, U, n_sites, z):
    intersite = None
    n_sites = x_dimension*y_dimension
    for i in range(1, n_sites+1):
        for j in range(1, n_sites+1):
            if i != j:
                vij = _ohno(unit_dist, U, i, j)
                ni, nj = _get_n(z,i, j)
                intersite = 0.5*vij*ni*nj
    return intersite 

def _ohno(unit_dist, U, i, j):
    rij = (j-i)*unit_dist
    vij = 14.397*(((28.794/2*U)**2)+rij**2)**0.5
    return vij

def _get_n(z,i, j):
    n_sites = x_dimension*y_dimension
    op_class = FermionOperator
    n_i = number_operator(n_sites, i)
    n_j = number_operator(n_sites, j)
    return (n_i-z, n_j-z)


def _right_neighbor(site, x_dimension, y_dimension, periodic):
    if x_dimension == 1:
        return None
    if (site + 1) % x_dimension == 0:
        if periodic:
            return site + 1 - x_dimension
        else:
            return None
    return site + 1


def _bottom_neighbor(site, x_dimension, y_dimension, periodic):
    if y_dimension == 1:
        return None
    if site + x_dimension + 1 > x_dimension * y_dimension:
        if periodic:
            return site + x_dimension - x_dimension * y_dimension
        else:
            return None
    return site + x_dimension

In [63]:
"""Define the Hamiltonian."""
# Parameters.
nsites = 4
U = 11.26
J = 2.4
z = 1
x_dimension = nsites
y_dimension = 1
unit_dist = 1.4
ppp = fermi_ppp(z,
                  unit_dist,
                  x_dimension,
                  y_dimension,
                  -J,
                  U,
                  chemical_potential=0.,
                  magnetic_field=0.,
                  periodic=True,
                  spinless=False,
                  particle_hole_symmetry=False)
print(ppp)

4667.975738930096 [] +
-11.26 [0^ 0] +
11.26 [0^ 0 1^ 1] +
2.4 [0^ 2] +
2.4 [0^ 6] +
2.4 [1^ 3] +
2.4 [1^ 7] +
2.4 [2^ 0] +
-11.26 [2^ 2] +
11.26 [2^ 2 3^ 3] +
2.4 [2^ 4] +
2.4 [3^ 1] +
-4667.975738930096 [3^ 3] +
2.4 [3^ 5] +
2.4 [4^ 2] +
-4679.235738930096 [4^ 4] +
4667.975738930096 [4^ 4 3^ 3] +
11.26 [4^ 4 5^ 5] +
2.4 [4^ 6] +
2.4 [5^ 3] +
2.4 [5^ 7] +
2.4 [6^ 0] +
2.4 [6^ 4] +
-11.26 [6^ 6] +
11.26 [6^ 6 7^ 7] +
2.4 [7^ 1] +
2.4 [7^ 5]
